In [1]:
from pathlib import Path

In [13]:
tv_dir_path =Path(r'D:\data\dpe_full\dpe_doc\tv_tables')
tr_dir_path =Path(r'D:\data\dpe_full\dpe_doc\tr_tables')

In [14]:
trtv_dir_path=tv_dir_path

In [15]:
    trtv_dir_path = Path(trtv_dir_path)
    trtv_table_dict = dict()
    for table_path in trtv_dir_path.iterdir():
        table_name = table_path.name.split('.csv')[0]
        table = pd.read_csv(table_path).astype('category')
        ids = [col for col in table if col.endswith('id')]
        table[ids] = table[ids].astype(str).astype('category')
        table_ref = table_path.name.split('_')[0]
        table.columns = [table_ref + '_' + el if not el.startswith(('tr', 'tv')) else el for el in table.columns]
        table = table.rename(columns={f'{table_ref}_id': f'{table_name}_id'})
        trtv_table_dict[table_name] = table

In [12]:
trtv_dir_path = Path(trtv_dir_path)


tv025_type_batiment not found
tv025_type_emetteur not found
tv025_equipement_intermittence not found
tv029_simu_type_distribution not found


In [15]:
def load_tvtr_table_dict(tvtr_dir_path):

    tvtr_dir_path = Path(tvtr_dir_path)
    tvtr_table_dict =dict()
    for table_path in tvtr_dir_path.iterdir():
        table_name =table_path.name.split('.csv')[0]
        table = pd.read_csv(table_path).astype('category')
        table_ref = table_path.name.split('_')[0]
        table.columns = [table_ref+'_'+el if not el.startswith(('tr','tvtr')) else el for el in table.columns ]
        table = table.rename(columns ={f'{table_ref}_id':f'{table_name}_id'})
        tvtr_table_dict[table_name] = table 
    for table_name,table in tvtr_table_dict.items():

        foreign_ids = [col for col in table if col.endswith('_id') and not col.startswith(table_name)]
        for foreign_id in foreign_ids:
            foreign_table_name = foreign_id.split('_id')[0]
            foreign_table=tvtr_table_dict.get(foreign_table_name,None)
            if foreign_table is None:
                print(foreign_table_name+' not found')
            else:
                table = table.merge(foreign_table,on = foreign_id,how='left')
        cols = [el for el in table if not any(s in el for s in ['simulateur','simu_ordre','id'])]
        cols.append(f'{table_name}_id')
        tvtr_table_dict[table_name] =table[cols]
    return tvtr_table_dict

In [26]:

class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class DPEMetaData(metaclass=Singleton):

    def __init__(self ,assets_dir=Path(r'D:\data\dpe_full\dpe_doc')):

        self.tv_table_dict = load_tv_table_dict(assets_dir/'tv_tables')
        self.tr_table_dict = load_tr_table_dict(assets_dir/'tr_tables')


    def merge_all_tr_table(self, table):
        table = table.copy()
        tr_cols = [col for col in table.columns if col.startswith('tr')]
        for tr_col in tr_cols:
            table = self.merge_tr_table(table, tr_col)
        return table



    def merge_all_tv_table(self, table):
        table = table.copy()
        tv_cols = [col for col in table.columns if col.startswith('tv')]
        for tv_col in tv_cols:
            table = self.merge_tv_table(table, tv_col)
        return table
    
    def merge_tv_table(self, table, tv_col):
        table.merge(self.tv_table_dict[])
        tv_table_name = tv_col.split('_id')[0]
        tv_table = self.tv_table_dict[tv_table_name]
        table = table.merge(tv_table, on=tv_col, how='left')
        table.index = table.index.astype(stv)
        cols = tv_table.columns.tolist()
        table[cols]=table[cols].astype('category')
        return table




In [23]:
td007

,id,td006_batiment_id,tr014_type_parois_opaque_id,reference,deperdition_thermique,tv001_coefficient_reduction_deperditions_id,tv002_local_non_chauffe_id,coefficient_transmission_thermique_paroi,coefficient_transmission_thermique_paroi_non_isolee,tv003_umur_id,tv004_umur0_id,tv005_upb_id,tv006_upb0_id,tv007_uph_id,tv008_uph0_id,resistance_thermique_isolation,epaisseur_isolation,surface_paroi
0,1048,209,1,Mur,40.27,1,NaN,0.35,0.00,NaN,78,NaN,NaN,NaN,NaN,0.00,10.00,115.06
1,1049,209,1,Mur,3.59,NaN,1,0.35,0.00,NaN,78,NaN,NaN,NaN,NaN,0.00,10.00,10.80
2,1050,209,4,Plafond,6.49,NaN,5,0.14,0.00,NaN,NaN,NaN,NaN,NaN,10,0.00,27.00,49.10
3,1814,364,1,Mur,31.71,1,NaN,0.38,0.00,1,78,NaN,NaN,NaN,NaN,0.00,9.00,82.67
4,1815,364,1,Mur,3.89,NaN,1,0.32,0.00,1,1,NaN,NaN,NaN,NaN,0.00,0.00,12.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439933,24115581,5419822,3,Plancher,17.35,261,NaN,0.25,0.00,NaN,NaN,85,9,NaN,NaN,0.00,0.00,69.40
439934,24115582,5419822,1,Mur,45.94,1,NaN,0.40,0.00,51,78,NaN,NaN,NaN,NaN,0.00,10.00,114.86
439935,24115588,5419822,1,Mur,5.60,NaN,1,0.40,0.00,51,69,NaN,NaN,NaN,NaN,0.00,0.00,17.50
439936,24115589,5419822,4,Plafond,9.33,1,NaN,0.23,0.00,NaN,NaN,NaN,NaN,99,10,0.00,0.00,40.55


In [1]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

In [2]:
from assets_orm import DPEMetaData
meta = DPEMetaData()

tv013_valeur_pont_thermique_isolation_planche_bas not found
tv025_type_batiment not found
tv025_type_emetteur not found
tv025_equipement_intermittence not found
tv026_classe_inertie_plancher_bas not found
tv026_classe_inertie_plancher_haut not found
tv026_classe_inertie_paroi_verticale not found
tv026_classe_inertie_classe_inertie not found
tv029_simu_type_distribution not found


In [17]:
td011 =pd.read_csv(data_dir/'depts'/dept/'td011_installation_chauffage.csv',dtype=str)
td012 =pd.read_csv(data_dir/'depts'/dept/'td012_generateur_chauffage.csv',dtype=str)
td013 =pd.read_csv(data_dir/'depts'/dept/'td013_installation_ecs.csv',dtype=str)
td014 =pd.read_csv(data_dir/'depts'/dept/'td014_generateur_ecs.csv',dtype=str)
td006 =pd.read_csv(data_dir/'depts'/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/'depts'/dept/'td001_dpe.csv',dtype=str)
td005 =pd.read_csv(data_dir/'depts'/dept/'td005_fiche_technique.csv',dtype=str)
td007 =pd.read_csv(data_dir/'depts'/dept/'td007_paroi_opaque.csv',dtype=str)
td006 =pd.read_csv(data_dir/'depts'/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/'depts'/dept/'td001_dpe.csv',dtype=str)
td008 =pd.read_csv(data_dir/'depts'/dept/'td008_baie.csv',dtype=str)

In [18]:
td008.dtypes.astype(str).to_dict()

{'id': 'object',
 'td007_paroi_opaque_id': 'object',
 'reference': 'object',
 'td008_baie_id': 'object',
 'deperdition': 'object',
 'tv009_coefficient_transmission_thermique_vitrage_id': 'object',
 'presence_survitrage': 'object',
 'coefficient_transmission_thermique_baie': 'object',
 'tv010_coefficient_transmission_thermique_baie_id': 'object',
 'tv011_resistance_additionnelle_id': 'object',
 'tv012_coef_transmission_thermique_baie_protection_solaire_id': 'object',
 'surface': 'object',
 'perimetre': 'object',
 'tv013_valeur_pont_thermique_id': 'object',
 'facteur_solaire': 'object',
 'tv021_facteur_solaire_id': 'object',
 'tv022_coefficient_masques_proches_id': 'object',
 'coefficient_masques_lointains_non_homogenes': 'object',
 'tv023_coefficient_masques_lointains_homogenes_id': 'object',
 'tv020_coefficient_orientation_id': 'object'}

In [7]:
dept = '38'

td007 =pd.read_csv(data_dir/'depts'/dept/'td007_paroi_opaque.csv',dtype=str)
td008 =pd.read_csv(data_dir/'depts'/dept/'td008_baie.csv',dtype=str)

In [8]:
td008_types = {'id': 'str',
               'td007_paroi_opaque_id': 'str',
               'reference': 'str',
               'td008_baie_id': 'str',
               'deperdition': 'float',
               'tv009_coefficient_transmission_thermique_vitrage_id': 'category',
               'presence_survitrage': 'str',
               'coefficient_transmission_thermique_baie': 'float',
               'tv010_coefficient_transmission_thermique_baie_id': 'category',
               'tv011_resistance_additionnelle_id': 'category',
               'tv012_coef_transmission_thermique_baie_protection_solaire_id': 'category',
               'surface': 'float',
               'perimetre': 'float',
               'tv013_valeur_pont_thermique_id': 'category',
               'facteur_solaire': 'float',
               'tv021_facteur_solaire_id': 'category',
               'tv022_coefficient_masques_proches_id': 'category',
               'coefficient_masques_lointains_non_homogenes': 'category',
               'tv023_coefficient_masques_lointains_homogenes_id': 'category',
               'tv020_coefficient_orientation_id': 'category'}


def merge_td008_tr_tv(td008):
    from assets_orm import DPEMetaData
    meta = DPEMetaData()
    table = td008.copy()
    table = table.rename(columns={
        'tv012_coef_transmission_thermique_baie_protection_solaire_id': 'tv012_coefficient_transmission_thermique_baie_protection_solaire_id'})
    table = meta.merge_all_tr_tables(table)

    table = meta.merge_all_tv_tables(table)

    table = table.astype({k: v for k, v in td008_types.items() if k in table})
    table = table.loc[:, ~table.columns.duplicated()]

    return table

In [9]:
td008 = merge_td008_tr_tv(td008)

In [27]:
td008['tv020_orientation_paroi'].value_counts()

Sud      144681
Est      107954
Ouest    106621
Nord     105858
Name: tv020_orientation_paroi, dtype: int64

In [30]:
td008['tv020_coefficient_orientation_id'].iloc[0]

'3'

In [18]:
td008['tv009_epaisseur_lame'].value_counts()

16    96933
12    49170
6     42385
0     18884
8     18457
20    16568
10    16359
14    15959
15    10534
18     9164
Name: tv009_epaisseur_lame, dtype: int64

In [31]:
td008.count().to_dict()

{'id': 527286,
 'td007_paroi_opaque_id': 527286,
 'reference': 527286,
 'td008_baie_id': 527286,
 'deperdition': 527286,
 'tv009_coefficient_transmission_thermique_vitrage_id': 294413,
 'presence_survitrage': 527286,
 'coefficient_transmission_thermique_baie': 527286,
 'tv010_coefficient_transmission_thermique_baie_id': 486742,
 'tv011_resistance_additionnelle_id': 389495,
 'tv012_coefficient_transmission_thermique_baie_protection_solaire_id': 162115,
 'surface': 527286,
 'perimetre': 527286,
 'tv013_valeur_pont_thermique_id': 527286,
 'facteur_solaire': 527286,
 'tv021_facteur_solaire_id': 475713,
 'tv022_coefficient_masques_proches_id': 445700,
 'coefficient_masques_lointains_non_homogenes': 509884,
 'tv023_coefficient_masques_lointains_homogenes_id': 279628,
 'tv020_coefficient_orientation_id': 527286,
 'tv009_code': 294413,
 'tv009_epaisseur_lame': 294413,
 'tv009_ug': 294413,
 'tv009_type_vitrage': 294413,
 'tv009_orientation': 275529,
 'tv009_remplissage': 275529,
 'tv009_traitem

In [25]:
td008['tv009_epaisseur_lame'].astype(pd.Int64Dtype()).astype(int).astype(str).replace('0', '')

TypeError: object cannot be converted to an IntegerDtype

In [ ]:
td008.tv022_coefficient_masques_proches_id

In [13]:
td008['tv009_type_vitrage'].value_counts()

Double Vitrage    274820
Simple Vitrage     18884
Triple Vitrage       709
Survitrage             0
Name: tv009_type_vitrage, dtype: int64

In [15]:
td008.tv009_epaisseur_lame.value_counts()

16    96933
12    49170
6     42385
0     18884
8     18457
20    16568
10    16359
14    15959
15    10534
18     9164
Name: tv009_epaisseur_lame, dtype: int64

In [31]:
meta.trtv_table_dict['tv022_coefficient_masques_proches']

,tv022_code,tv022_rapport_l1_l2,tv022_largeur_baie_superieure,tv022_beta_gama,tv022_angle_superieur_30,tv022_fe1,tv022_type_masque,tv022_avance,tv0xx_orientation,tv022_coefficient_masques_proches_id
0,TV022_001,NaN,NaN,NaN,NaN,1.00,Absence de masque proche,NaN,NaN,1
1,TV022_002,NaN,NaN,NaN,NaN,0.40,Fond de balcon ou fond et flanc de loggias,< 1 m,Nord,2
2,TV022_003,NaN,NaN,NaN,NaN,0.50,Fond de balcon ou fond et flanc de loggias,< 1 m,Sud,3
3,TV022_004,NaN,NaN,NaN,NaN,0.45,Fond de balcon ou fond et flanc de loggias,< 1 m,Est ou Ouest,4
4,TV022_005,NaN,NaN,NaN,NaN,0.30,Fond de balcon ou fond et flanc de loggias,1 <= … < 2,Nord,5
5,TV022_006,NaN,NaN,NaN,NaN,0.40,Fond de balcon ou fond et flanc de loggias,1 <= … < 2,Sud,6
6,TV022_007,NaN,NaN,NaN,NaN,0.35,Fond de balcon ou fond et flanc de loggias,1 <= … < 2,Est ou Ouest,7
7,TV022_008,NaN,NaN,NaN,NaN,0.20,Fond de balcon ou fond et flanc de loggias,2 <= … < 3,Nord,8
8,TV022_009,NaN,NaN,NaN,NaN,0.30,Fond de balcon ou fond et flanc de loggias,2 <= … < 3,Sud,9
9,TV022_010,NaN,NaN,NaN,NaN,0.25,Fond de balcon ou fond et flanc de loggias,2 <= … < 3,Est ou Ouest,10


In [22]:
list(meta.trtv_table_dict.keys())

['tv001_coefficient_reduction_deperditions',
 'tv002_local_non_chauffe',
 'tv002_local_non_chauffe_type_batiment',
 'tv003_umur',
 'tv004_umur0',
 'tv004_umur0_materiaux',
 'tv005_upb',
 'tv006_upb0',
 'tv007_uph',
 'tv007_uph_type_toit',
 'tv008_uph0',
 'tv009_coefficient_transmission_thermique_vitrage',
 'tv009_ug_orientation',
 'tv009_ug_remplissage',
 'tv009_ug_traitement_vitrage',
 'tv009_ug_type_vitrage',
 'tv010_coefficient_transmission_thermique_baie',
 'tv010_uw_type_baie',
 'tv010_uw_type_materiaux',
 'tv010_uw_type_menuiserie',
 'tv011_resistance_additionnelle',
 'tv012_coefficient_transmission_thermique_baie_protection_solaire',
 'tv013_valeur_pont_thermique',
 'tv013_valeur_pont_thermique_isolation_mur',
 'tv013_valeur_pont_thermique_isolation_plancher_bas',
 'tv013_valeur_pont_thermique_retour_isolation',
 'tv013_valeur_pont_thermique_type_liaison',
 'tv013_valeur_pont_thermique_type_pose',
 'tv014_permeabilite',
 'tv015_valeur_conventionnelle_renouvellement_air',
 'tv016

In [20]:
td012.columns

Index(['id', 'systeme_chauffage_cogeneration_id',
       'td011_installation_chauffage_id', 'tr004_type_energie_id',
       'tv045_conversion_kwh_co2_id',
       'tv046_evaluation_contenu_co2_reseaux_id',
       'rendement_emission_systeme_chauffage',
       'tv028_rendement_emission_systeme_chauffage_id',
       'rendement_distribution_systeme_chauffage',
       'tv029_rendement_distribution_systeme_chauffage_id',
       'tv030_rendement_regulation_systeme_chauffage_id',
       'rendement_generation', 'tv031_rendement_generation_id',
       'presence_regulation', 'coefficient_performance',
       'tv032_coefficient_performance_id',
       'tv033_coefficient_correction_regulation_id',
       'tv034_temperature_fonctionnement_chaudiere_100_id',
       'tv035_temperature_fonctionnement_chaudiere_30_id', 'rpn', 'rpint',
       'qp0', 'puissance_veilleuse', 'tv036_puissance_veilleuse_id',
       'puissance_nominale', 'tv038_puissance_nominale_id',
       'consommation_chauffage'],
      dt

In [13]:
td007 = meta.merge_all_tv_tables(td007)
td007 = meta.merge_all_tr_tables(td007)

KeyError: 'tv001_code'

In [12]:
meta.trtv_table_dict['tv003_umur'].count()

tv003_code                      64
tv003_mur_isole                 22
tv003_annee_construction        45
tv003_annee_construction_min    45
tv003_annee_construction_max    45
tv003_annee_isolation           18
tv003_annee_isolation_min       18
tv003_annee_isolation_max       18
tv003_effet_joule               42
tv003_umur                      64
tv017_code                       0
tv017_t_ext_moyen                0
tv017_peta_cw                    0
tv017_dh14                       0
tv017_prs1                       0
tv003_umur_id                   64
dtype: int64

In [5]:
td007.tv017_code.value_counts()

H1    130370
H3         0
H2         0
Name: tv017_code, dtype: int64

In [16]:
td007['tr014_sous_type'].value_counts()

Paroi verticale extérieure    245457
Plancher bas                   82832
Plancher haut                  82716
Paroi verticale intérieure     21495
Plancher intermédiaire          7438
Name: tr014_sous_type, dtype: int64

In [14]:
td007.count().sort_values(ascending=False).to_dict()

{'tr014_sous_type': 439938,
 'tr014_code': 439938,
 'td006_batiment_id': 439938,
 'tr014_type_parois_opaque_id': 439938,
 'deperdition_thermique': 439938,
 'coefficient_transmission_thermique_paroi': 439938,
 'surface_paroi': 439938,
 'id': 439938,
 'reference': 439926,
 'epaisseur_isolation': 435416,
 'resistance_thermique_isolation': 429240,
 'tv001_code': 361729,
 'tv001_aiu_aue': 361729,
 'tv001_valeur': 361729,
 'tv001_coefficient_reduction_deperditions_id': 361729,
 'coefficient_transmission_thermique_paroi_non_isolee': 335701,
 'tv004_umur0_id': 243518,
 'tv004_epaisseur': 243518,
 'tv004_code': 243518,
 'tv004_materiaux': 243518,
 'tv004_umur': 243518,
 'tv003_umur_id': 191403,
 'tv003_code': 191403,
 'tv003_umur': 191403,
 'tv003_annee_construction_max': 134474,
 'tv003_annee_construction_min': 134474,
 'tv003_annee_construction': 134474,
 'tv017_code': 130370,
 'tv017_peta_cw': 130370,
 'tv017_dh14': 130370,
 'tv017_prs1': 130370,
 'tv017_t_ext_moyen': 130370,
 'tv002_type_ba

In [23]:
tv004=meta.trtv_table_dict['tv004_umur0']

In [11]:
td007.tr014_type_parois_opaque_id.iloc[0]

'1'

In [12]:
td007.columns.sort_values()

Index(['coefficient_transmission_thermique_paroi',
       'coefficient_transmission_thermique_paroi_non_isolee',
       'deperdition_thermique', 'epaisseur_isolation', 'id', 'reference',
       'resistance_thermique_isolation', 'surface_paroi', 'td006_batiment_id',
       'tr014_code', 'tr014_sous_type', 'tr014_type_parois_opaque_id',
       'tv001_aiu_aue', 'tv001_aiu_aue_max', 'tv001_aiu_aue_min',
       'tv001_aiu_isole', 'tv001_aue_isole', 'tv001_code',
       'tv001_coefficient_reduction_deperditions_id', 'tv001_uv_ue',
       'tv001_valeur', 'tv002_code', 'tv002_local_non_chauffe',
       'tv002_local_non_chauffe_id', 'tv002_type_batiment', 'tv002_uvue',
       'tv003_annee_construction', 'tv003_annee_construction_max',
       'tv003_annee_construction_min', 'tv003_annee_isolation',
       'tv003_annee_isolation_max', 'tv003_annee_isolation_min', 'tv003_code',
       'tv003_effet_joule', 'tv003_mur_isole', 'tv003_umur', 'tv003_umur_id',
       'tv004_code', 'tv004_epaisseur', 'tv

In [31]:
tv004.tv004_umur0_id.iloc[0]

1

In [29]:
td007.merge(tv004,on = 'tv004_umur0_id')

,id,td006_batiment_id,tr014_type_parois_opaque_id,reference,deperdition_thermique,tv001_coefficient_reduction_deperditions_id,tv002_local_non_chauffe_id,coefficient_transmission_thermique_paroi,coefficient_transmission_thermique_paroi_non_isolee,tv003_umur_id,...,tv008_uph0,tv008_est_efface,tr014_code,tr014_sous_type,tr014_est_efface,tv004_code_y,tv004_epaisseur_y,tv004_umur_y,tv004_est_efface_y,tv004_materiaux_y


In [5]:
td007.columns

NameError: name 'table' is not defined

In [10]:
foreign_ids

[]

In [10]:
td007.tr014_type_parois_opaque_id.iloc[0]

'1'

In [16]:
td007.tv006_code.value_counts().sort_values()

TV006_012    0
TV006_011    0
TV006_010    0
TV006_009    0
TV006_008    0
TV006_007    0
TV006_006    0
TV006_005    0
TV006_004    0
TV006_003    0
TV006_002    0
TV006_001    0
Name: tv006_code, dtype: int64

In [8]:
td007.tv007_annee_construction.value_counts()

à partir de 2006    0
à partir de 2000    0
jusqu'à 1988        0
de 1989 à 2000      0
2000 à 2005         0
1988 à 2000         0
1982 à 1988         0
1977 à 1982         0
1974 à 1977         0
1948 à 1974         0
Name: tv007_annee_construction, dtype: int64

In [ ]:
meta.trtv_table_di